In [1]:
import torch
torch.cuda.is_available()

True

In [2]:
import pandas as pd

df_train = pd.read_csv('../dataset/train/train.csv')
df_test = pd.read_csv('../dataset/test/test_data.csv')

In [ ]:
import ast


def replace_string_index(sentence,subj_start,subj_end,subj_type,subj_word,obj_start,obj_end,obj_type,obj_word):
    tag_subj=f"<S:{subj_type}> {subj_word} </S:{subj_type}>"
    tag_obj=f"<O:{obj_type}> {obj_word} </O:{obj_type}>"

    if subj_start<obj_start:
        return sentence[:subj_start] + tag_subj + sentence[subj_end+1:obj_start] + tag_obj + sentence[obj_end+1:]
    else:
        return sentence[:obj_start] + tag_obj + sentence[obj_end+1:subj_start] + tag_subj + sentence[subj_end+1:]


def get_typed_entity_marker(df):
    """
    모든 label에 대해 Typed Entity Marker 진행
    """

    sentence, label = df['sentence'].tolist(),df['label'].tolist()
    subject_entity, object_entity = df['subject_entity'].tolist(),df['object_entity'].tolist()
    sentence_masked=[]

    for i in range(len(label)):
        dict_subj, dict_obj = ast.literal_eval(subject_entity[i]),ast.literal_eval(object_entity[i])
        subj_start,subj_end,subj_type,subj_word = dict_subj['start_idx'],dict_subj['end_idx'],dict_subj['type'],dict_subj['word']
        obj_start,obj_end,obj_type,obj_word = dict_obj['start_idx'],dict_obj['end_idx'],dict_obj['type'],dict_obj['word']

        sentence_masked.append(replace_string_index(sentence[i],subj_start,subj_end,subj_type,subj_word,
                                                    obj_start,obj_end,obj_type,obj_word))

    return sentence_masked

In [ ]:
sentence_masked=get_typed_entity_marker(df_train)

df_train_tem=df_train.copy()
df_train_tem['sentence']=sentence_masked

In [ ]:
import os
import sys

cwd = os.getcwd()
sys.path.append(os.path.abspath(os.path.join(cwd, os.pardir)))

In [ ]:
from utils import train_valid_split

df_train_em,df_test_em=train_valid_split(df_train_tem)
df_train_em.to_csv('../dataset/train/train-v.0.1.1.csv')
df_train_em.to_csv('../dataset/dev/dev-v.0.1.1.csv')